In [1]:
import pandas as pd 
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import dash 
from dash import dcc, html, Input, Output
import logging
import os
import requests

In [2]:
def get_stock_data(ticker, years=1):
    start_date=(datetime.datetime.today() - datetime.timedelta(days=365*years)).strftime('%Y-%m-%d')
    end_date=datetime.datetime.today().strftime('%Y-%m-%d')
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

In [4]:
# Alpha Vantage API Key
API_KEY = os.environ['ALPHAVANTAGE_API_KEY']

# Define the URL for the stock listing endpoint
url = f'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={API_KEY}'

# Fetch stock listings
response = requests.get(url)
data = response.text

# Save to a CSV file
with open('listing_status.csv', 'w') as f:
    f.write(data)

# Read the CSV into a DataFrame for further processing
symbols_df = pd.read_csv('listing_status.csv')

# Filter for NASDAQ and NYSE
nasdaq_nyse_df = symbols_df[symbols_df['exchange'].isin(['NASDAQ', 'NYSE'])]

# Save filtered data to a CSV file
nasdaq_nyse_df.to_csv('nasdaq_nyse_symbols.csv', index=False)

print("NASDAQ and NYSE symbols saved to CSV.")

NASDAQ and NYSE symbols saved to CSV.


In [5]:
ticker_list = pd.read_csv('nasdaq_nyse_symbols.csv')
ticker_list.head(10)

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaN,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaN,Active
2,AACG,ATA Creativity Global,NASDAQ,Stock,2008-01-29,NaN,Active
3,AACT,Ares Acquisition Corporation II - Class A,NYSE,Stock,2023-06-12,NaN,Active
4,AACT-U,Ares Acquisition Corporation II - Units (1 Ord...,NYSE,Stock,2023-04-21,NaN,Active
5,AACT-WS,Ares Acquisition Corporation II - Warrants (01...,NYSE,Stock,2023-06-12,NaN,Active
6,AADI,Aadi Bioscience Inc,NASDAQ,Stock,2017-08-08,NaN,Active
7,AADR,ADVISORSHARES DORSEY WRIGHT ADR ETF,NASDAQ,ETF,2010-07-21,NaN,Active
8,AAGR,African Agriculture Holdings Inc,NASDAQ,Stock,2023-12-07,NaN,Active
9,AAGRW,African Agriculture Holdings Inc - Warrants (0...,NASDAQ,Stock,2023-12-07,NaN,Active


In [3]:
df = get_stock_data('SPY', years=5)

[*********************100%***********************]  1 of 1 completed


In [10]:
app = dash.Dash(__name__)


app.layout = html.Div([

    html.H1("Stock Price Analysis Dashboard", style={'text-align': 'center'}),
    html.Div([
    html.H2("Enter the stock symbol:"),
    dcc.Input(id='input', value='SPY', type='text')
    ]),
    dcc.Graph(id='output-graph',
             figure={'data': [
                    {'x': df.index, 'y': df['Close'], 'type': 'line', 'name': 'stock'},
                ],'layout': {
                    'title': 'S&P 500 Index',
                    'xaxis': {'title': 'Date'},
                    'yaxis': {'title': 'Price'}
                }})
])

@app.callback(
    Output('output-graph', 'figure'),
    Input('input', 'value')
)
def update_graph(stock_ticker):
    # Get the stock data for the selected stock if it is not empty
    # Use error handling, in case the stock does not exist
    if not stock_ticker:
        return {
            'data': [
                {'x': [], 'y': [], 'type': 'line', 'name': 'stock'},
            ],
            'layout': {
                'title': 'Stock',
                'xaxis': {'title': 'Date'},
                'yaxis': {'title': 'Price'}
            }
        }
    elif not yf.Ticker(stock_ticker).history(period='1d').empty:
        df = get_stock_data(stock_ticker, years=5)
        return {
            'data': [
                {'x': df.index, 'y': df['Close'], 'type': 'line', 'name': 'stock'},
            ],
            'layout': {
                'title': f'Closing Price for {stock_ticker}',
                'xaxis': {'title': 'Date'},
                'yaxis': {'title': 'Price'}
            }
        }
    else:
        return {
            'data': [
                {'x': [], 'y': [], 'type': 'line', 'name': 'stock'},
            ],
            'layout': {
                'title': 'Stock',
                'xaxis': {'title': 'Date'},
                'yaxis': {'title': 'Price'}
            }
        }
  

if __name__ == '__main__':
    app.run_server(debug=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
$JP: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
[*********************100%***********************]  1 of 1 completed
$JP: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['JP']: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
[****************

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[10], line 40, in update_graph(stock_ticker='BA')
     29     return {
     30         'data': [
     31             {'x': [], 'y': [], 'type': 'line', 'name': 'stock'},
   (...)
     37         }
     38     }
     39 elif not yf.Ticker(stock_ticker).history(period='1d').empty:
---> 40     df = get_stock_data(stock_ticker, years=5)
        stock_ticker = 'BA'
        df =                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2019-09-10  297.359985  298.200012  295.970001  298.130005  275.551056   
2019-09-11  298.470001  300.339996  297.750000  300.250000  277.510498   
2019-09-12  301.250000  302.459991  300.410004  301.290009  278.471771   
2019-09-13  301.779999  302.170013  300.679993  301.089996  278.286926   
2019-09-16  299.83999

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
$N: possibly delisted; no price data found  (period=1d)
$NV: possibly delisted; no price data found  (period=1d)
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 